# Spatial Patterns of Change


## Imports and Functions


In [1]:
# Data wrangling
import pandas as pd
import numpy as np
import geopandas as gpd
import h3pandas

# Data visualisation
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pylab import f

# My functions
import sys

sys.path.insert(0, "../../src")
from run_mp import *
from utilities import *
from random_forest_utils import *

# Other
from os import error
import warnings
import chime
from pyprojroot import here

chime.theme("mario")

# Magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

## User Input


In [18]:
force_rerunning_all_sites = False
run_only_subset = True
subset_fraction = 0.10

## Load Data


In [3]:
# Load all shapefiles
shp_cty = gpd.read_file(here("data/raw/maps/france_geojson/cty.geojson"))
shp_reg = gpd.read_file(here("data/raw/maps/france_geojson/reg.geojson"))
shp_dep = gpd.read_file(here("data/raw/maps/france_geojson/dep.geojson"))
shp_gre = gpd.read_file(here("data/raw/maps/france_geojson/gre.geojson"))
shp_ser = gpd.read_file(here("data/raw/maps/france_geojson/ser.geojson"))
shp_hex = load_hexmap()

In [4]:
# Load NFI data
nfi_raw = pd.read_feather(here("data/tmp/nfi/nfi_ready_for_analysis.feather"))
nfi_raw.shape

(549255, 193)

## Calculate Change at IDP-Level for All Subsets


In [16]:
# * Get input df --------------------------------------------------------------
df_in = nfi_raw.copy()
if run_only_subset:
    all_sites = df_in["idp"].unique().tolist()
    subset_sites = np.random.choice(
        all_sites, int(len(all_sites) * subset_fraction), replace=False
    ).tolist()
    df_in = df_in.query("idp in @subset_sites")

# * Get factorial setup -------------------------------------------------------
# ! Pick only idp level here but loop over all species and heights
all_regions = ["idp"]  # ["idp", "reg", "dep", "gre", "ser", "hex"]

# Pick top 10 species + populus
all_species = (
    df_in["genus_lat"]
    .value_counts()
    .sort_values(ascending=False)
    .head(10)
    .index.tolist()
)
all_species = ["all"] + all_species
if "Populus" not in all_species:
    all_species.append("Populus")

# Pick all height classes
all_heights = ["all"] + df_in["tree_height_class"].value_counts().index.tolist()
all_heights.remove("Missing")

# * Run loop ------------------------------------------------------------------
# Imports
from utilities import calculate_growth_mortality_optimized
from run_mp import run_mp

# Loop over species
for my_species in all_species:
    # Loop over heights
    for my_height in all_heights:
        # Get new df
        df_loop = df_in.copy()

        # ! Filter df
        # Filter for species
        if my_species != "all":
            df_loop = df_loop.query(f"genus_lat == '{my_species}'")

        # Filter for height
        if my_height != "all":
            df_loop = df_loop.query(f"tree_height_class == '{my_height}'")

        # Loop over regions
        for my_region in all_regions:
            # ! Get filename and print it
            if run_only_subset:
                my_dir = here(
                    str.lower(
                        f"data/tmp/nfi/spatio-temporal-analysis/idp/subset_{round(subset_fraction*100)}%_of_sites"
                    )
                )
            else:
                my_dir = here(str.lower(f"data/tmp/nfi/spatio-temporal-analysis/idp/"))
            os.makedirs(my_dir, exist_ok=True)
            my_file = str.lower(f"species_{my_species}-height_{my_height}")
            my_dirfile = f"{my_dir}/{my_file}.feather"

            # Check if file already exists, if so skip current loop
            if os.path.isfile(my_dirfile) and not force_rerunning_all_sites:
                print(f" - File already exists: {my_file}, skipping it")
                continue

            # Verbose
            print(str.lower(f"\n - Working on: {my_file}"))

            # ! Add grouping variable
            my_region_tmp = my_region

            # IDP holds year information, so no need to add there
            # But grouping for other regions need this time information
            if my_region != "idp":
                my_region = my_region + "_year"
                df_loop[my_region] = (
                    df_loop[my_region_tmp].astype(str)
                    + "_"
                    + df_loop["campagne_1"].astype(str)
                )

            # ! Create list and run mp
            df_list = [
                group for name, group in df_loop.groupby(my_region, as_index=False)
            ]
            df_mp = run_mp(
                calculate_growth_mortality_optimized,
                df_list,
                combine_func=pd.concat,
                progress_bar=True,
                num_cores=10,
                grouping_variable=my_region,
            )

            # ! Save df
            df_mp.reset_index(drop=True).to_feather(my_dirfile)


 - working on: species_all-height_all


100%|██████████| 4023/4023 [09:36<00:00,  6.98it/s]



 - working on: species_all-height_10-15


100%|██████████| 2620/2620 [06:25<00:00,  6.79it/s]



 - working on: species_all-height_0-10


...

100%|██████████| 107/107 [00:19<00:00,  5.52it/s]



 - working on: species_populus-height_20-25


100%|██████████| 116/116 [00:20<00:00,  5.77it/s]



 - working on: species_populus-height_25+


100%|██████████| 122/122 [00:21<00:00,  5.78it/s]


## Directly at regional level


In [29]:
#
# * Get input df --------------------------------------------------------------
df_in = nfi_raw.copy()
if run_only_subset:
    all_sites = df_in["idp"].unique().tolist()
    subset_sites = np.random.choice(
        all_sites, int(len(all_sites) * subset_fraction), replace=False
    ).tolist()
    df_in = df_in.query("idp in @subset_sites")

# * Get factorial setup -------------------------------------------------------
# ! Loop over all regions (not idp) but not over all heights, would take too long for now...
all_regions = ["reg", "dep", "gre", "ser", "hex"]

# Pick top 10 species + populus
all_species = (
    df_in["genus_lat"]
    .value_counts()
    .sort_values(ascending=False)
    .head(10)
    .index.tolist()
)
all_species = ["all"] + all_species
if "Populus" not in all_species:
    all_species.append("Populus")

# Pick all height classes
# + df_in["tree_height_class"].value_counts().index.tolist()all_heights.remove("Missing")
all_heights = ["all"]

# * Run loop ------------------------------------------------------------------
# Imports
from utilities import calculate_growth_mortality_optimized
from run_mp import run_mp

# Loop over species
for my_species in all_species:
    # Loop over heights
    for my_height in all_heights:
        # Get new df
        df_loop = df_in.copy()

        # ! Filter df
        # Filter for species
        if my_species != "all":
            df_loop = df_loop.query(f"genus_lat == '{my_species}'")

        # Filter for height
        if my_height != "all":
            df_loop = df_loop.query(f"tree_height_class == '{my_height}'")

        # Loop over regions
        for my_region in all_regions:
            # ! Get filename and print it
            if run_only_subset:
                my_dir = here(
                    str.lower(
                        f"data/tmp/nfi/spatio-temporal-analysis/direct/subset_{round(subset_fraction*100)}%_of_sites"
                    )
                )
            else:
                my_dir = here(
                    str.lower(f"data/tmp/nfi/spatio-temporal-analysis/direct/")
                )
            os.makedirs(my_dir, exist_ok=True)
            my_file = str.lower(
                f"species_{my_species}-height_{my_height}_region-{my_region}"
            )
            my_dirfile = f"{my_dir}/{my_file}.feather"

            # Check if file already exists, if so skip current loop
            if os.path.isfile(my_dirfile) and not force_rerunning_all_sites:
                print(f" - File already exists: {my_file}, skipping it")
                continue

            # Verbose
            print(str.lower(f"\n - Working on: {my_file}"))

            # ! Add grouping variable
            my_region_tmp = my_region

            # IDP holds year information, so no need to add there
            # But grouping for other regions need this time information
            if my_region != "idp":
                my_region = my_region + "_year"
                df_loop[my_region] = (
                    df_loop[my_region_tmp].astype(str)
                    + "_"
                    + df_loop["campagne_1"].astype(str)
                )

            # ! Create list and run mp
            df_list = [
                group for name, group in df_loop.groupby(my_region, as_index=False)
            ]

            df_mp = run_mp(
                calculate_growth_mortality_optimized,
                df_list,
                combine_func=pd.concat,
                progress_bar=True,
                num_cores=10,
                grouping_variable=my_region,
            )

            # ! Save df
            df_mp.reset_index(drop=True).to_feather(my_dirfile)


 - working on: species_all-height_all_region-reg


100%|██████████| 91/91 [00:18<00:00,  4.81it/s]



 - working on: species_all-height_all_region-dep


100%|██████████| 625/625 [01:43<00:00,  6.06it/s]



 - working on: species_all-height_all_region-gre


...

100%|██████████| 55/55 [00:11<00:00,  4.71it/s]



 - working on: species_populus-height_all_region-ser


100%|██████████| 217/217 [00:35<00:00,  6.05it/s]



 - working on: species_populus-height_all_region-hex


100%|██████████| 314/314 [00:49<00:00,  6.35it/s]


## Spatial Patterns

- There are two approaches:

  - Direct: Aggregate trees to given area and calculate change _directly_ at this level
    - To assess the uncertainty per area and year, this approach requires a bootstrapping
  - Aggregated: Aggregate trees to site and take the mean over the given area
    - To assess the uncertainty per area and year, this approach simply reports the mean and sd across sites

- Plus, we are interested in all species and in species subsets.


### Direct Approach


In [15]:
# Imports
from utilities import calculate_growth_mortality
from run_mp import run_mp

# Get factorial setup
all_regions = ["idp", "reg", "dep", "gre", "ser", "hex"]
all_species = ["all"] + nfi_raw["genus_lat"].value_counts().sort_values(ascending=False)
all_species = all_species.head(10).index.tolist()
all_heights = ["all"] + df_merged["tree_height_class"].value_counts().index.tolist()
all_heights.remove("Missing")

# Make sure poplars are in there
if "Populus" not in all_species:
    all_species.append("Populus")

# Get input df
df_in = nfi_raw.copy()

# Loop over species
for my_species in all_species:
    # Loop over heights
    for my_height in all_heights:
        # Get new df
        df_loop = df_in.copy()

        # Filter for species
        if my_species != "all":
            df_loop = df_loop.query(f"genus_lat == '{my_species}'")

        # Filter for height
        if my_height != "all":
            df_loop = df_loop.query(f"tree_height_class == '{my_height}'")

        # Loop over regions
        for my_region in all_regions:
            # Get filename and print it
            my_dir = here(
                str.lower(f"data/tmp/nfi/spatio-temporal-analysis/direct_approach")
            )
            os.makedirs(my_dir, exist_ok=True)
            my_file = str.lower(f"species_{my_species}-area_{my_region}")
            my_dirfile = f"{my_dir}/{my_file}.feather"

            # Check if file already exists, if so skip current loop
            if os.path.isfile(my_dirfile) and not force_rerunning_all_sites:
                print(f" - File already exists: {my_file}, skipping it")
                continue

            # Verbose
            print(str.lower(f"\n - Working on: {my_file}"))

            my_region_tmp = my_region

            # IDP holds year information, so no need to add there
            # But grouping for other regions need this time information
            if my_region != "idp":
                my_region = my_region + "_year"
                df_loop[my_region] = (
                    df_loop[my_region_tmp].astype(str)
                    + "_"
                    + df_loop["campagne_1"].astype(str)
                )

            # Create list and run mp
            df_list = [
                group for name, group in df_loop.groupby(my_region, as_index=False)
            ]
            df_mp = run_mp(
                calculate_growth_mortality,
                df_list,
                combine_func=pd.concat,
                progress_bar=True,
                num_cores=10,
                min_trees_per_plot=1,
                grouping_variable=my_region,
            )

            # Save df
            df_mp.reset_index(drop=True).to_feather(my_dirfile)

 - File already exists: species_all-area_idp, skipping it

 - working on: species_all-area_reg


100%|██████████| 91/91 [00:24<00:00,  3.73it/s]



 - working on: species_all-area_dep


100%|██████████| 649/649 [02:20<00:00,  4.61it/s]



 - working on: species_all-area_gre


...

100%|██████████| 602/602 [02:22<00:00,  4.23it/s]



 - working on: species_all-area_hex


100%|██████████| 1582/1582 [05:11<00:00,  5.09it/s]



 - working on: species_quercus-area_idp


 91%|█████████▏| 20905/22900 [59:34<22:21,  1.49it/s] 

### Correct Approach


In [ ]:
# Load the idp-level data for each species and calculate the mean and sd over desired region

## Temporal Patterns

- For the temporal patterns, we can again do the holistic or the mean-approach:
- For now, to save time, I am only doing the mean approach


### Mean-Approach

- For this, I need factorial species-height subsets but only at the idp level


In [ ]:
# Calculate metrics of change at region-level
all_regions = ["idp"]
all_species = ["all"] + df_merged["genus_lat"].value_counts().head(10).index.tolist()
all_heights = ["all"] + df_merged["tree_height_class"].value_counts().index.tolist()
all_heights.remove("Missing")

from utilities import calculate_growth_mortality
from run_mp import run_mp

df_in = df_merged.copy()
group_per_height = True

for my_species in all_species:
    for my_height in all_heights:
        # Get new df
        df_loop = df_in.copy()
        # Filter for species
        if my_species != "all":
            df_loop = df_loop.query(f"genus_lat == '{my_species}'")
        # Filter for height
        if my_height != "all":
            df_loop = df_loop.query(f"tree_height_class == '{my_height}'")

        for my_region in all_regions:
            my_file = str.lower(
                f"species_{my_species}__tree_heights_{my_height}__area_{my_region}"
            )

            print(str.lower(f"\n - Working on: {my_file}"))

            my_region_tmp = my_region

            # IDP holds year information, so no need to add there
            # But grouping for other regions need this information
            if my_region != "idp":
                my_region = my_region + "_year"
                df_loop[my_region] = (
                    df_loop[my_region_tmp].astype(str)
                    + "_"
                    + df_loop["campagne_1"].astype(str)
                )

            # Create list and run mp
            df_list = [
                group for name, group in df_loop.groupby(my_region, as_index=False)
            ]
            df_mp = run_mp(
                calculate_growth_mortality,
                df_list,
                combine_func=pd.concat,
                progress_bar=True,
                num_cores=10,
                min_trees_per_plot=1,
                grouping_variable=my_region,
            )

            # Save df
            my_dir = here(str.lower(f"data/tmp/dfs_metrics_of_change/area-level/"))
            if not os.path.exists(my_dir):
                os.makedirs(my_dir)
            df_mp.reset_index(drop=True).to_feather(f"{my_dir}/{my_file}.feather")

---

## Load Data


### Maps


In [6]:
shp_hex10 = gpd.read_file(here("qgis/hexmaps/hexmap_10000.geojson"))
shp_hex12 = gpd.read_file(here("qgis/hexmaps/hexmap_12500.geojson"))
shp_hex25 = gpd.read_file(here("qgis/hexmaps/hexmap_25000.geojson"))

DriverError: /Users/pascal/repos/padasch/ifn_analysis/qgis/hexmaps/hexmap_10000.geojson: No such file or directory

### NFI Data


In [2]:
# Load NFI data ready for analysis
data = pd.read_feather("nfi_ready_for_spatial_temporal_analysis.feather")

# Move ser and gre columns to the start
data.insert(0, "ser", data.pop("ser"))
data.insert(0, "gre", data.pop("gre"))
data.insert(0, "lat", data.pop("lat"))
data.insert(0, "lon", data.pop("lon"))
data.insert(0, "campagne_1", data.pop("campagne_1"))
data.insert(0, "idp", data.pop("idp"))
data.head()

,idp,campagne_1,lon,lat,gre,ser,hdec,ddec,sfpied,sfdorge_1,...,tform_1,plisi,tpespar1_1,prelev5,videplant,incid_1,lat_fr,lon_fr,greco,reg
0,632691,2011,-2.842824,48.337505,A,A11,10.8,NaN,0,Missing,...,NaN,0.0,73,NaN,NaN,0.0,6.820144e+06,267415.027897,A,53
1,632691,2011,-2.842824,48.337505,A,A11,18.4,NaN,0,Missing,...,NaN,0.0,73,NaN,NaN,0.0,6.820144e+06,267415.027897,A,53
2,632691,2011,-2.842824,48.337505,A,A11,13.0,NaN,0,Missing,...,NaN,0.0,73,NaN,NaN,0.0,6.820144e+06,267415.027897,A,53
3,632691,2011,-2.842824,48.337505,A,A11,10.4,NaN,0,Missing,...,NaN,0.0,73,NaN,NaN,0.0,6.820144e+06,267415.027897,A,53
4,632691,2011,-2.842824,48.337505,A,A11,21.0,NaN,0,Missing,...,NaN,0.0,73,NaN,NaN,0.0,6.820144e+06,267415.027897,A,53


In [13]:
# Create dictionary of genus-espar
dict_species = data[["genus_lat", "espar_red"]].drop_duplicates().reset_index(drop=True)
dict_species["espar_red"].value_counts(ascending=True).tail(10)

espar_red
28     1
27     1
3      1
41     1
25     2
57     2
29     7
23     9
68    11
49    19
Name: count, dtype: int64

In [16]:
dict_species.query("espar_red == '68'")

,genus_lat,espar_red
0,Chamaecyparis,68
70,Pinus,68
79,Taxodium,68
81,Cupressus,68
82,Thuja,68
87,Tsuga,68
89,Sequoia,68
99,Cedrus,68
100,Conifer,68
102,Cryptomeria,68


In [17]:
dict_species.query("genus_lat == 'Abies'")

,genus_lat,espar_red
23,Abies,61
71,Abies,72
73,Abies,71
97,Abies,70
108,Abies,68


### Metric of Change


In [9]:
# Calculate on site-level for aggregation to larger region later on
df_list = data.groupby("idp", as_index=False)
df_list = [group for name, group in df_list]
df_site = run_mp(
    calculate_growth_mortality,
    df_list,
    combine_func=pd.concat,
    progress_bar=True,
    num_cores=10,
    divide_by_nplots=True,
    grouping_variable=my_group,
)

  8%|▊         | 3055/40221 [06:49<1:23:06,  7.45it/s]


KeyboardInterrupt: 

In [ ]:
# Regional Level
df_list = data.groupby("reg", as_index=False)
df_list = [group for name, group in df_list]
df_reg = run_mp(
    calculate_growth_mortality,
    df_list,
    combine_func=pd.concat,
    progress_bar=True,
    num_cores=10,
    grouping_variable="reg",
    divide_by_nplots=True,
)

In [ ]:
# Department Level
df_list = data.groupby("dep", as_index=False)
df_list = [group for name, group in df_list]
df_dep = run_mp(
    calculate_growth_mortality,
    df_list,
    combine_func=pd.concat,
    progress_bar=True,
    num_cores=10,
    grouping_variable="dep",
    divide_by_nplots=True,
)

In [ ]:
# Sylvoregion Level
df_list = data.groupby("ser", as_index=False)
df_list = [group for name, group in df_list]
df_ser = run_mp(
    calculate_growth_mortality,
    df_list,
    combine_func=pd.concat,
    progress_bar=True,
    num_cores=10,
    grouping_variable="ser",
    divide_by_nplots=True,
)

In [ ]:
# Greco Level
df_list = data.groupby("gre", as_index=False)
df_list = [group for name, group in df_list]
df_gre = run_mp(
    calculate_growth_mortality,
    df_list,
    combine_func=pd.concat,
    progress_bar=True,
    num_cores=10,
    grouping_variable="gre",
    divide_by_nplots=True,
)

## Calculate Metrics of Change

**Target:** Mean of Metric within Region

Routine:

- Filter for tree subset
- For each site, calculate metrics of change
- Group sites by given variable (ser, gre, dep, reg, hex)
- Calculate the mean and sd for that group
- Plot mean and sd for that group

**Alternative:** Calculate total change per region (aggregate all sites to one instead of taking the mean across sites).

**Maps:**

- Plot The difference relative to 2010 Campagnes!


### All species


### Top 10 Species


---
